In [1]:
import sys
sys.path.append('../OOSpotify/')
from OOSpotify import *
import pickle
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

Using TensorFlow backend.
/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def prepSequenceData(track,maxlen=4163):
    aa = track.getAudioAnalysis()
    df = pd.DataFrame(aa['segments'])
    pitches = np.array([i for i in df.pitches])
    timbres = np.array([i for i in df.timbre])
    loudness = np.repeat(df.loudness_max.tolist(),12).reshape(-1,12)
    duration = np.repeat(df.duration.tolist(),12).reshape(-1,12)
    confidence = np.repeat(df.confidence.tolist(),12).reshape(-1,12)
    sequence_data = np.concatenate((pitches,timbres,loudness,duration,confidence),axis=1)
    sequence_data_padded = pad_sequences([sequence_data],maxlen=maxlen)
    return sequence_data_padded

In [3]:
def prepMetaData(track):
    features = track.features.items()
    return np.array([val for key,val in features]).reshape(-1,1)

In [4]:
def extractTracks(SpotifyObjs):
    if type(SpotifyObjs) is not list:
        SpotifyObjs = [SpotifyObjs]
    tracklist = []
    for obj in SpotifyObjs:
        if obj.type == 'track':
            tracklist.append(obj)
        elif obj.type == 'artist':
            tracks = obj.getTopTracks()
            tracklist += tracks
            print('Using top {} tracks for artist {}'.format(len(tracks),obj.name))
        elif obj.type in ['playlist','album']:
            tracks = obj.getTracks()
            tracklist += tracks
        else:
            #invalid type
            pass
    return tracklist
        
def prepData(SpotifyObjs):
    seq = []
    meta = []
    tracklist = extractTracks(SpotifyObjs)
    for i,track in enumerate(tracklist):
            print('{}. {} -- {}'.format(i,track.name,track.getArtist().name))
            seq.append(prepSequenceData(track))
            meta.append(prepMetaData(track))
    seq = np.squeeze(np.array(seq))
    meta = np.array(meta)
    print()
    print('Sequence Shape: {}'.format(seq.shape))
    print('Metadata Shape: {}'.format(meta.shape))
    tracklist = [track.name for track in tracklist]
    return seq,meta,tracklist

In [37]:
seq,meta,tracklist = prepData([Album('mr. finish line'),Track('back pocket')])

0. Birds of a Feather, We Rock Together (feat. Antwaun Stanley) -- Vulfpeck
1. Baby I Don’t Know Oh Oh (feat. Charles Jones) -- Vulfpeck
2. Mr. Finish Line (feat. Christine Hucal & Theo Katzman) -- Vulfpeck
3. Tee Time -- Vulfpeck
4. Running Away (feat. Joey Dosik, David T. Walker & James Gadson) -- Vulfpeck
5. Hero Town (feat. Michael Bland) -- Vulfpeck
6. Business Casual (feat. Coco O.) -- Vulfpeck
7. Vulf Pack -- Vulfpeck
8. Grandma (feat. Antwaun Stanley, David T. Walker & James Gadson) -- Vulfpeck
9. Captain Hook (feat. Baby Theo, Bootsy Collins & Mushy Kay) -- Vulfpeck
10. Back Pocket -- Vulfpeck

Sequence Shape: (11, 4163, 60)
Metadata Shape: (11, 12, 1)


In [7]:
model = load_model('weights.05--2.44.hdf5')

ValueError: bad marshal data (unknown type code)

In [39]:
a = model.predict(x=[seq,meta.reshape(-1,12)])

In [6]:
genreList = pickle.load(open('genreList.p','rb'))
for i,g in enumerate(genreList):
    print(i,g)

0 Alternative Rock
1 Bluegrass
2 Blues
3 Classical
4 Country
5 Dubstep
6 EDM
7 Electronic
8 Emo
9 Folk
10 Gospel
11 Grunge
12 Hip Hop
13 Indie Rock
14 Jazz
15 Metal
16 Motown
17 Pop
18 Rap
19 Reggae
20 Rock
21 Ska Punk
22 Soundtrack


In [41]:
['{} --> {}'.format(tracklist[i],genreList[np.argmax(a[0][i])]) for i in range(11)]

['Birds of a Feather, We Rock Together (feat. Antwaun Stanley) --> Folk',
 'Baby I Don’t Know Oh Oh (feat. Charles Jones) --> Pop',
 'Mr. Finish Line (feat. Christine Hucal & Theo Katzman) --> Hip Hop',
 'Tee Time --> Electronic',
 'Running Away (feat. Joey Dosik, David T. Walker & James Gadson) --> Hip Hop',
 'Hero Town (feat. Michael Bland) --> Electronic',
 'Business Casual (feat. Coco O.) --> Hip Hop',
 'Vulf Pack --> Electronic',
 'Grandma (feat. Antwaun Stanley, David T. Walker & James Gadson) --> Hip Hop',
 'Captain Hook (feat. Baby Theo, Bootsy Collins & Mushy Kay) --> Hip Hop',
 'Back Pocket --> Pop']

In [54]:
seq,meta,tracklist = prepData([Album('an awesome wave')])

0. Intro -- alt-J
1. Interlude 1 -- alt-J
2. Tessellate -- alt-J
3. Breezeblocks -- alt-J
4. Interlude 2 -- alt-J
5. Something Good -- alt-J
6. Dissolve Me -- alt-J
7. Matilda -- alt-J
8. Ms -- alt-J
9. Fitzpleasure -- alt-J
10. Interlude 3 -- alt-J
11. Bloodflood -- alt-J
12. Taro -- alt-J
13. Hand-Made -- alt-J

Sequence Shape: (14, 4163, 60)
Metadata Shape: (14, 12, 1)


In [55]:
a = model.predict(x=[seq,meta.reshape(-1,12)])
b = ['{} --> {}'.format(tracklist[i],genreList[np.argmax(a[0][i])]) for i in range(len(a[0]))]
print(b)

['Intro --> Electronic', 'Interlude 1 --> Hip Hop', 'Tessellate --> Pop', 'Breezeblocks --> Indie Rock', 'Interlude 2 --> Classical', 'Something Good --> Folk', 'Dissolve Me --> EDM', 'Matilda --> Indie Rock', 'Ms --> Folk', 'Fitzpleasure --> Indie Rock', 'Interlude 3 --> Classical', 'Bloodflood --> Indie Rock', 'Taro --> Folk', 'Hand-Made --> Folk']


In [61]:
seq,meta,tracklist = prepData([Artist('bob dylan').getAlbums()[0]])

0. Slow Train - Live in London -- Bob Dylan
1. Precious Angel - Live at the Warfield Theatre, San Francisco, CA - November 16, 1979 -- Bob Dylan
2. Gotta Serve Somebody - Live at the Warfield Theatre, San Francisco, CA - November 15, 1979 -- Bob Dylan
3. When He Returns - Take 2 -- Bob Dylan
4. Solid Rock - Live at Golden Hall, San Diego, CA - November 27, 1979 -- Bob Dylan
5. Gonna Change My Way of Thinking - Live at the Orpheum Theater, Memphis, TN - January 31, 1980 -- Bob Dylan
6. Ain't Gonna Go to Hell for Anybody - Live at Le Theatre Saint-Denis, Montreal, Quebec - April 24, 1980 -- Bob Dylan
7. When You Gonna Wake Up? - Live at Drammenshallen, Drammen, Norway - July 9, 1981 -- Bob Dylan
8. Covenant Woman - Take 3 -- Bob Dylan
9. Shot of Love - Outtake -- Bob Dylan
10. Making a Liar Out of Me - Rehearsal -- Bob Dylan
11. City of Gold - Live Nov. 22, 1980 -- Bob Dylan
12. Yonder Comes Sin - Rehearsal -- Bob Dylan
13. Caribbean Wind - Rehearsal with pedal steel -- Bob Dylan
14. Dea

In [62]:
a = model.predict(x=[seq,meta.reshape(-1,12)])
for i in range(len(a[0])):
    print('{} --> {},{}'.format(tracklist[i],genreList[np.argmax(a[0][i])],genreList[np.argmax(a[1][i])]))
#b = ['{} --> {},{}'.format(tracklist[i],genreList[np.argmax(a[0][i])],genreList[np.argmax(a[1][i])]) for i in range(len(a[0]))]
#print(b)

Slow Train - Live in London --> Rock,Rock
Precious Angel - Live at the Warfield Theatre, San Francisco, CA - November 16, 1979 --> Rock,Classical
Gotta Serve Somebody - Live at the Warfield Theatre, San Francisco, CA - November 15, 1979 --> Indie Rock,Indie Rock
When He Returns - Take 2 --> Jazz,Motown
Solid Rock - Live at Golden Hall, San Diego, CA - November 27, 1979 --> Pop,Classical
Gonna Change My Way of Thinking - Live at the Orpheum Theater, Memphis, TN - January 31, 1980 --> Motown,Indie Rock
Ain't Gonna Go to Hell for Anybody - Live at Le Theatre Saint-Denis, Montreal, Quebec - April 24, 1980 --> Rock,Indie Rock
When You Gonna Wake Up? - Live at Drammenshallen, Drammen, Norway - July 9, 1981 --> Jazz,Classical
Covenant Woman - Take 3 --> Jazz,Motown
Shot of Love - Outtake --> Motown,Motown
Making a Liar Out of Me - Rehearsal --> Rock,Rock
City of Gold - Live Nov. 22, 1980 --> Folk,Indie Rock
Yonder Comes Sin - Rehearsal --> Rock,Rock
Caribbean Wind - Rehearsal with pedal steel

In [8]:
model = load_model('bs60.30--2.04.hdf5')

ValueError: bad marshal data (unknown type code)